In [6]:
import pandas as pd
import math

In [7]:
df = pd.read_excel('p table.xlsx')

In [8]:
df

,crop_name,Zr,p
0,Rice,0.5-1.0,0.20
1,Sesame,1.0-1.5,0.60
2,Sunflower,0.8-1.5,0.45
3,Chick pea,0.6-1.0,0.50
4,Beans,0.5-0.7,0.45
5,Potato,0.4-0.6,0.35
6,Watermelon,0.8-1.5,0.40
7,Cucumber,0.7-1.2,0.50
8,Pumpkin,1.0-1.5,0.35
9,Egg Plant,0.7-1.2,0.45


In [13]:
def calculate_water_requirements(silt, clay, rsm, crop_name, rooting_depth=None):
    # Step 1: Compute BD (g/cm³)
    BD = 1.66 - 0.063 * math.log10(clay + 1)

    # Step 2: Compute θ_fc (field capacity) in m³/m³
    theta_fc_vol_percent = 0.2576 + 0.00204 * clay + 0.00322 * silt
    theta_fc = theta_fc_vol_percent / 100  # convert to m³/m³

    # Step 3: Compute θ_wp (wilting point) in m³/m³
    theta_wp_vol_percent = 0.026 + 0.005 * clay + 0.0158 * silt
    theta_wp = theta_wp_vol_percent / 100  # convert to m³/m³

    # Step 4: Get crop-specific info (p and Zr) from the DataFrame
    crop_row = df[df['crop_name'].str.lower() == crop_name.lower()]
    if crop_row.empty:
        print(f"Crop '{crop_name}' not found in the data.")
        return

    p = float(crop_row['p'].values[0])
    #print(p)
    Zr_val = crop_row['Zr'].values[0]
    if rooting_depth is None:
        if isinstance(Zr_val, str) and '-' in Zr_val:
            Zr_range = list(map(float, Zr_val.split('-')))
            Zr = sum(Zr_range) / len(Zr_range)  # take average
        else:
            Zr = float(Zr_val)
    else:
        Zr = rooting_depth
    #print(Zr)

    # Step 5: Calculate TAW (mm)
    TAW = 1000 * (theta_fc - theta_wp) * BD * Zr

    # Step 6: Calculate RAW (mm)
    RAW = p * TAW

    # Step 7: Decide irrigation
    irrigation_needed = rsm < RAW

    # Output
    print(f"\n--- Results for Crop: {crop_name} ---")
    print(f"Bulk Density (BD): {BD:.3f} g/cm³")
    print(f"Field Capacity (θ_fc): {theta_fc:.4f} m³/m³")
    print(f"Wilting Point (θ_wp): {theta_wp:.4f} m³/m³")
    print(f"Total Available Water (TAW): {TAW:.2f} mm")
    print(f"Readily Available Water (RAW): {RAW:.2f} mm")
    print(f"Root Zone Soil Moisture (RSM): {rsm} mm")
    print("Decision: TURN ON the pump 🚰" if irrigation_needed else "Decision: No need to irrigate ❌")


In [17]:
# Example input
silt = 13.2     # in %
clay = 6.6     # in %/
rsm = 0.144      # in % so divide by 100
crop_name = "egg plant"  # should match name in Excel

# Call the function
calculate_water_requirements(silt, clay, rsm, crop_name)


--- Results for Crop: egg plant ---
Bulk Density (BD): 1.605 g/cm³
Field Capacity (θ_fc): 0.0031 m³/m³
Wilting Point (θ_wp): 0.0027 m³/m³
Total Available Water (TAW): 0.70 mm
Readily Available Water (RAW): 0.32 mm
Root Zone Soil Moisture (RSM): 0.144 mm
Decision: TURN ON the pump 🚰
